In [ ]:
# Good reference: https://www.tensorflow.org/guide/performance/datasets
# Another good reference: https://www.tensorflow.org/guide/datasets
# Might be able to store all data in memory, but will implement at first as though this is not an option

In [ ]:
import numpy as np
import tensorflow as tf
from dataset.severstal_steel_dataset import SeverstalSteelDataset
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dataset = SeverstalSteelDataset('SETTINGS.yaml')

In [ ]:
data = dataset.create_dataset(training=True)
iterator = data.make_one_shot_iterator()
img, ann = iterator.get_next()

In [ ]:
with tf.Session() as sess:
    batch = 0
    while True:
        try:
            img_out, ann_out = sess.run((img, ann))
            break
        except tf.errors.OutOfRangeError:
            print('End of dataset.')
            break
        
        batch += 1
        print(f'batch: {batch}')


In [ ]:
img_out.shape

In [ ]:
ann_out.shape

In [ ]:
plt.imshow(np.repeat(img_out[0, :, :, :], 3, axis=-1))
#plt.imshow(ann[2, :, :, 2])
plt.show()

In [ ]:
plt.imshow(ann_out[0, :, :, 3])
plt.show()